<a href="https://colab.research.google.com/github/eluisluzquadros/covid_impact_se/blob/main/step01_2_collection_merge_grey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
SEMPRE RODAR ANTES

In [1]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [2]:
# Manipulação e visualização de dados
import os
import sys
import glob
import numpy as np
import pandas as pd


#dataviz
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output 

from google.colab import drive

# Definimos o parâmetro abaixo para evitar o erro:
# RecursionError: maximum recursion depth exceeded
# Ao gravar os resultados em disco
sys.setrecursionlimit(50000)

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


### Connect google drive

In [3]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impactSE/data/01_grey/2022Sample/step01_collect/en/"

In [5]:
# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

2021-12-31.csv	2022-01-01.csv


# Input

## Merge csv

In [6]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [7]:
len(all_filenames)

2

In [8]:
#combine all files in the list
df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])

In [9]:
print(df.shape);df.head(2)

(83, 22)


,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,...,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,IpAddress
0,0,2021-12-31,Congratulating Achievements - IWR ANNUAL AWARD...,"('Dec 31, 2021',)",https://www.iwr.usace.army.mil/Media/News-Stor...,https,www.iwr.usace.army.mil,/Media/News-Stories/Article/2886389/congratula...,NaN,NaN,...,US,United States,Georgia,Atlanta,-84.388283,33.798458,6453,2022-08-03 12:37:19.335859,geolocation,104.105.228.146
1,1,2021-12-31,Senior Software Development Engineer (Fully Re...,"('Dec 31, 2021',)",https://www.builtinnyc.com/job/engineer/senior...,https,www.builtinnyc.com,/job/engineer/senior-software-development-engi...,NaN,NaN,...,US,United States,California,San Jose,-121.838226,37.330528,13335,2022-08-03 12:37:19.335912,geolocation,104.17.112.12


In [10]:
df['status1-duplicated'] = df.duplicated(subset=['header','summary','link'])

In [11]:
df['status1-duplicated'].value_counts()

False    69
True     14
Name: status1-duplicated, dtype: int64

In [12]:
df['status2-fit_criteria'] = ''
df['status3-relevance_by_title-abst_keys'] = ''

In [13]:
df = df.sort_values(by=['date','Unnamed: 0'], ascending=True)
df = df.reset_index()

In [14]:
print(df.shape); df.head()

(83, 26)


,index,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,...,City,Longitude,Latitude,Asn,TimeGenerated,Type,IpAddress,status1-duplicated,status2-fit_criteria,status3-relevance_by_title-abst_keys
0,0,0,2021-12-31,Congratulating Achievements - IWR ANNUAL AWARD...,"('Dec 31, 2021',)",https://www.iwr.usace.army.mil/Media/News-Stor...,https,www.iwr.usace.army.mil,/Media/News-Stories/Article/2886389/congratula...,NaN,...,Atlanta,-84.388283,33.798458,6453,2022-08-03 12:37:19.335859,geolocation,104.105.228.146,False,,
1,1,1,2021-12-31,Senior Software Development Engineer (Fully Re...,"('Dec 31, 2021',)",https://www.builtinnyc.com/job/engineer/senior...,https,www.builtinnyc.com,/job/engineer/senior-software-development-engi...,NaN,...,San Jose,-121.838226,37.330528,13335,2022-08-03 12:37:19.335912,geolocation,104.17.112.12,False,,
2,2,2,2021-12-31,Case Studies | VividCloud | Cloud Design + Dev...,"('Dec 31, 2021',)",https://www.vividcloud.com/vividcloud-design-d...,https,www.vividcloud.com,/vividcloud-design-development,NaN,...,Kansas City,-94.539612,39.107712,396982,2022-08-03 12:37:19.335932,geolocation,34.117.168.233,False,,
3,3,3,2021-12-31,Cyber Security Risks And Challenges In Remote ...,"('Dec 31, 2021',)",https://www.europeanproceedings.com/article/10...,https,www.europeanproceedings.com,/article/10.15405/epsbs.2021.12.04.2,NaN,...,Tuusula,25.011320,60.426418,24940,2022-08-03 12:37:19.335950,geolocation,95.216.48.137,False,,
4,4,4,2021-12-31,December Major Update of ImmuniWeb Discovery,"('Jan 1, 2022',)",https://logon-int.com/blog/december-major-upda...,https,logon-int.com,/blog/december-major-update-of-immuniweb-disco...,NaN,...,Kwai Chung,114.130257,22.366850,7540,2022-08-03 12:37:19.335967,geolocation,202.181.185.163,False,,


In [15]:
df = df.reset_index()

In [16]:
print(df.shape); df.head()

(83, 27)


,level_0,index,Unnamed: 0,date,header,summary,link,protocol,domain,path,...,City,Longitude,Latitude,Asn,TimeGenerated,Type,IpAddress,status1-duplicated,status2-fit_criteria,status3-relevance_by_title-abst_keys
0,0,0,0,2021-12-31,Congratulating Achievements - IWR ANNUAL AWARD...,"('Dec 31, 2021',)",https://www.iwr.usace.army.mil/Media/News-Stor...,https,www.iwr.usace.army.mil,/Media/News-Stories/Article/2886389/congratula...,...,Atlanta,-84.388283,33.798458,6453,2022-08-03 12:37:19.335859,geolocation,104.105.228.146,False,,
1,1,1,1,2021-12-31,Senior Software Development Engineer (Fully Re...,"('Dec 31, 2021',)",https://www.builtinnyc.com/job/engineer/senior...,https,www.builtinnyc.com,/job/engineer/senior-software-development-engi...,...,San Jose,-121.838226,37.330528,13335,2022-08-03 12:37:19.335912,geolocation,104.17.112.12,False,,
2,2,2,2,2021-12-31,Case Studies | VividCloud | Cloud Design + Dev...,"('Dec 31, 2021',)",https://www.vividcloud.com/vividcloud-design-d...,https,www.vividcloud.com,/vividcloud-design-development,...,Kansas City,-94.539612,39.107712,396982,2022-08-03 12:37:19.335932,geolocation,34.117.168.233,False,,
3,3,3,3,2021-12-31,Cyber Security Risks And Challenges In Remote ...,"('Dec 31, 2021',)",https://www.europeanproceedings.com/article/10...,https,www.europeanproceedings.com,/article/10.15405/epsbs.2021.12.04.2,...,Tuusula,25.011320,60.426418,24940,2022-08-03 12:37:19.335950,geolocation,95.216.48.137,False,,
4,4,4,4,2021-12-31,December Major Update of ImmuniWeb Discovery,"('Jan 1, 2022',)",https://logon-int.com/blog/december-major-upda...,https,logon-int.com,/blog/december-major-update-of-immuniweb-disco...,...,Kwai Chung,114.130257,22.366850,7540,2022-08-03 12:37:19.335967,geolocation,202.181.185.163,False,,


In [17]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [18]:
print(df.shape); df.head()

(83, 26)


,level_0,index,date,header,summary,link,protocol,domain,path,query,...,City,Longitude,Latitude,Asn,TimeGenerated,Type,IpAddress,status1-duplicated,status2-fit_criteria,status3-relevance_by_title-abst_keys
0,0,0,2021-12-31,Congratulating Achievements - IWR ANNUAL AWARD...,"('Dec 31, 2021',)",https://www.iwr.usace.army.mil/Media/News-Stor...,https,www.iwr.usace.army.mil,/Media/News-Stories/Article/2886389/congratula...,NaN,...,Atlanta,-84.388283,33.798458,6453,2022-08-03 12:37:19.335859,geolocation,104.105.228.146,False,,
1,1,1,2021-12-31,Senior Software Development Engineer (Fully Re...,"('Dec 31, 2021',)",https://www.builtinnyc.com/job/engineer/senior...,https,www.builtinnyc.com,/job/engineer/senior-software-development-engi...,NaN,...,San Jose,-121.838226,37.330528,13335,2022-08-03 12:37:19.335912,geolocation,104.17.112.12,False,,
2,2,2,2021-12-31,Case Studies | VividCloud | Cloud Design + Dev...,"('Dec 31, 2021',)",https://www.vividcloud.com/vividcloud-design-d...,https,www.vividcloud.com,/vividcloud-design-development,NaN,...,Kansas City,-94.539612,39.107712,396982,2022-08-03 12:37:19.335932,geolocation,34.117.168.233,False,,
3,3,3,2021-12-31,Cyber Security Risks And Challenges In Remote ...,"('Dec 31, 2021',)",https://www.europeanproceedings.com/article/10...,https,www.europeanproceedings.com,/article/10.15405/epsbs.2021.12.04.2,NaN,...,Tuusula,25.011320,60.426418,24940,2022-08-03 12:37:19.335950,geolocation,95.216.48.137,False,,
4,4,4,2021-12-31,December Major Update of ImmuniWeb Discovery,"('Jan 1, 2022',)",https://logon-int.com/blog/december-major-upda...,https,logon-int.com,/blog/december-major-update-of-immuniweb-disco...,NaN,...,Kwai Chung,114.130257,22.366850,7540,2022-08-03 12:37:19.335967,geolocation,202.181.185.163,False,,


In [19]:
df.rename(columns={'index':'page_rank','level_0':'index'}, inplace=True)

In [20]:
print(df.shape); df.head()

(83, 26)


,index,page_rank,date,header,summary,link,protocol,domain,path,query,...,City,Longitude,Latitude,Asn,TimeGenerated,Type,IpAddress,status1-duplicated,status2-fit_criteria,status3-relevance_by_title-abst_keys
0,0,0,2021-12-31,Congratulating Achievements - IWR ANNUAL AWARD...,"('Dec 31, 2021',)",https://www.iwr.usace.army.mil/Media/News-Stor...,https,www.iwr.usace.army.mil,/Media/News-Stories/Article/2886389/congratula...,NaN,...,Atlanta,-84.388283,33.798458,6453,2022-08-03 12:37:19.335859,geolocation,104.105.228.146,False,,
1,1,1,2021-12-31,Senior Software Development Engineer (Fully Re...,"('Dec 31, 2021',)",https://www.builtinnyc.com/job/engineer/senior...,https,www.builtinnyc.com,/job/engineer/senior-software-development-engi...,NaN,...,San Jose,-121.838226,37.330528,13335,2022-08-03 12:37:19.335912,geolocation,104.17.112.12,False,,
2,2,2,2021-12-31,Case Studies | VividCloud | Cloud Design + Dev...,"('Dec 31, 2021',)",https://www.vividcloud.com/vividcloud-design-d...,https,www.vividcloud.com,/vividcloud-design-development,NaN,...,Kansas City,-94.539612,39.107712,396982,2022-08-03 12:37:19.335932,geolocation,34.117.168.233,False,,
3,3,3,2021-12-31,Cyber Security Risks And Challenges In Remote ...,"('Dec 31, 2021',)",https://www.europeanproceedings.com/article/10...,https,www.europeanproceedings.com,/article/10.15405/epsbs.2021.12.04.2,NaN,...,Tuusula,25.011320,60.426418,24940,2022-08-03 12:37:19.335950,geolocation,95.216.48.137,False,,
4,4,4,2021-12-31,December Major Update of ImmuniWeb Discovery,"('Jan 1, 2022',)",https://logon-int.com/blog/december-major-upda...,https,logon-int.com,/blog/december-major-update-of-immuniweb-disco...,NaN,...,Kwai Chung,114.130257,22.366850,7540,2022-08-03 12:37:19.335967,geolocation,202.181.185.163,False,,


# GEOINT

In [21]:
import plotly.express as px 
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", zoom = 0)
fig.update_layout(mapbox_style="open-street-map") 
#fig.show()


# build jupyter dash app 
app = JupyterDash(__name__)
# add html components and figure to app
app.layout = html.Div([dcc.Graph(figure=fig)])
# run app inline
app.run_server(mode='inline')

<IPython.core.display.Javascript object>

### Get features from dates

In [22]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [23]:
df['year'] = df.date.dt.year
df['month'] = df.date.dt.month
df['quarter'] = df.date.dt.quarter
df['day'] = df.date.dt.day
df['day_of_week'] = df.date.dt.dayofweek

In [24]:
print(df.shape);
df[['date', 'header','year','quarter','day_of_week']].head(2)

(83, 31)


,date,header,year,quarter,day_of_week
0,2021-12-31,Congratulating Achievements - IWR ANNUAL AWARD...,2021,4,4
1,2021-12-31,Senior Software Development Engineer (Fully Re...,2021,4,4


# Output

In [25]:
path2 = "/content/drive/My Drive/covid19impactSE/data/01_grey/2022Sample/step02_screening/"

In [26]:
df.to_csv(path2 + 'step01-2_collection_merge_grey', sep=';', encoding='UTF-8')